In [ ]:
# Technical Indicators Implementation

import pandas as pd
import pandas_datareader.data as pdr
import datetime as dt
import numpy as np
from stocktrends import Renko
#import statsmodels.api as sm
#import statsmodels.regression.linear_model as srl


ticker = 'SPY'
ohlcv = pdr.get_data_yahoo(ticker, dt.date.today() - dt.timedelta(365), dt.date.today())
def MACD(dataframe, fast, slow, signal):
    df = dataframe.copy()
    df['Ma Fast'] = df['Adj Close'].ewm(span = slow, min_periods = slow).mean()
    df['Ma Slow'] = df['Adj Close'].ewm(span = fast, min_periods = fast).mean()
    df['MACD'] = df['Ma Fast'] - df['Ma Slow']
    df['Signal'] = df['MACD'].ewm(span = signal, min_periods = signal).mean()
    df.dropna(inplace = True)
    return df
#Graphical visualisation
#df.iloc[:,[8, 9]].plot(figsize = (30, 30), title = 'MACD AND SIGNAL')

def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2

def BollingerBands(dataframe, n):
    df = dataframe.copy()
    df['MA'] = df['Adj Close'].rolling(n).mean()
    df['BB_up'] = df['MA'] + df['MA'].rolling(n).std()
    df['BB_down'] = df['MA'] - df['MA'].rolling(n).std()
    df['BB_width'] = df['BB_up'] - df['BB_down']
    df.dropna(inplace = True)
    return df

def RSI(dataframe, n):
    df = dataframe.copy()
    df['Delta'] = df['Adj Close'] - df['Adj Close'].shift(1)
    df['Gain'] = np.where(df['Delta'] >= 0, df['Delta'], 0)
    df['Loss'] = np.where(df['Delta'] < 0, abs(df['Delta']), 0)
    avg_gain = []
    avg_loss = []
    gain = df['Gain'].tolist()
    loss = df['Loss'].tolist()
    
    for i in range(len(df)):
        
        if i < n:
            avg_gain.append(np.NaN)
            avg_loss.append(np.NaN)
        elif i == n:
            avg_gain.append(df['Gain'].rolling(n).mean().tolist()[n])
            avg_loss.append(df['Loss'].rolling(n).mean().tolist()[n])
        else:
            avg_gain.append(((n - 1) * avg_gain[i - 1] + gain[i])/n)
            avg_loss.append(((n - 1) * avg_loss[i - 1] + loss[i])/n)
    
    df['avg_gain'] = np.array(avg_gain)
    df['avg_loss'] = np.array(avg_loss)
    df['RS'] = df['avg_gain']/df['avg_loss']
    df['RSI'] = 100 - (100/(1 + df['RS']))
    df.dropna(inplace = True)
    return df['RSI']

def ADX(DF,n):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['TR'] = ATR(df2,n)['TR'] #the period parameter of ATR function does not matter because period does not influence TR calculation
    df2['DMplus']=np.where((df2['High']-df2['High'].shift(1))>(df2['Low'].shift(1)-df2['Low']),df2['High']-df2['High'].shift(1),0)
    df2['DMplus']=np.where(df2['DMplus']<0,0,df2['DMplus'])
    df2['DMminus']=np.where((df2['Low'].shift(1)-df2['Low'])>(df2['High']-df2['High'].shift(1)),df2['Low'].shift(1)-df2['Low'],0)
    df2['DMminus']=np.where(df2['DMminus']<0,0,df2['DMminus'])
    TRn = []
    DMplusN = []
    DMminusN = []
    TR = df2['TR'].tolist()
    DMplus = df2['DMplus'].tolist()
    DMminus = df2['DMminus'].tolist()
    for i in range(len(df2)):
        if i < n:
            TRn.append(np.NaN)
            DMplusN.append(np.NaN)
            DMminusN.append(np.NaN)
        elif i == n:
            TRn.append(df2['TR'].rolling(n).sum().tolist()[n])
            DMplusN.append(df2['DMplus'].rolling(n).sum().tolist()[n])
            DMminusN.append(df2['DMminus'].rolling(n).sum().tolist()[n])
        elif i > n:
            TRn.append(TRn[i-1] - (TRn[i-1]/14) + TR[i])
            DMplusN.append(DMplusN[i-1] - (DMplusN[i-1]/14) + DMplus[i])
            DMminusN.append(DMminusN[i-1] - (DMminusN[i-1]/14) + DMminus[i])
    df2['TRn'] = np.array(TRn)
    df2['DMplusN'] = np.array(DMplusN)
    df2['DMminusN'] = np.array(DMminusN)
    df2['DIplusN']=100*(df2['DMplusN']/df2['TRn'])
    df2['DIminusN']=100*(df2['DMminusN']/df2['TRn'])
    df2['DIdiff']=abs(df2['DIplusN']-df2['DIminusN'])
    df2['DIsum']=df2['DIplusN']+df2['DIminusN']
    df2['DX']=100*(df2['DIdiff']/df2['DIsum'])
    ADX = []
    DX = df2['DX'].tolist()
    for j in range(len(df2)):
        if j < 2*n-1:
            ADX.append(np.NaN)
        elif j == 2*n-1:
            ADX.append(df2['DX'][j-n+1:j+1].mean())
        elif j > 2*n-1:
            ADX.append(((n-1)*ADX[j-1] + DX[j])/n)
    df2['ADX']=np.array(ADX)
    return df2['ADX']

def OBV(dataframe):
    df = dataframe.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['direction'] = np.where(df['daily_ret']>=0, 1, -1)
    df['direction'][0] = 0
    df['vol_adj'] = df['Volume'] * df['direction']
    df['OBV'] = df['vol_adj'].cumsum()
    return df['OBV']

# not working due to import of statsmodels
def slope(dataframe, n):
    df = dataframe.copy()
    series = df['Adj Close']
    slopes = [i*0 for i in range(n-1)]
    for i in range(n, len(series) + 1):
        y = series[n-i:i]
        x = np.array(range(n))
        x_scaled = (x - x.min())/(x.max() - x.min())
        y_scaled = (y - y.min())/(y.max() - y.min())
        x_scaled = srl.add_constant(x_scaled)
        model = srl.OLS(y_scaled, x_scaled)
        results = model.fit()
        slopes.append(results.params[-1])
        slope_angle = (np.rad2deg(np.arctan(np.array(slopes))))
        return np.array(slope_angles)

def Renko_Trend(dataframe):
    df = dataframe.copy()
    df.reset_index(inplace = True)
    #df = df.iloc[:,[0,1,2,3,5,6]]
    #df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    df.rename(columns = {"Date" : "date", "High" : "high","Low" : "low", "Open" : "open","Close" : "close", "Volume" : "volume"}, inplace = True)
    renko_df = Renko(df)
    renko_df.brick_size = 5
    df2 = renko_df.get_ohlc_data()
    return df2

def CAGR(dataframe):
    df = dataframe.copy()
    df['Daily_Ret'] = df['Adj Close'].pct_change()
    df['Cum_Ret'] = (1+df['Daily_Ret']).cumprod()
    n = len(df)/252
    CAGR = (df['Cum_Ret'][-1])**(1/n) - 1
    return CAGR

def volatility(dataframe):
    df = dataframe.copy()
    df['Daily_Ret'] = df['Adj Close'].pct_change()
    vol = (df['Daily_Ret'].std())*(np.sqrt(252))
    return vol
    
rf = 0.016

def sharpe(dataframe, rf):
    df = dataframe.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr

def sortino(dataframe):
    df = dataframe.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    vol = (df[df['daily_ret']<0]['daily_ret'].std())*(np.sqrt(252))
    sr = (CAGR(df) - rf)/vol
    return sr
    
def max_dd(dataframe):
    df = dataframe.copy()
    df['daily_ret'] = df['Adj Close'].pct_change()
    df['cum_ret'] = (1+df['daily_ret']).cumprod()
    df['cum_roll_max'] = df['cum_ret'].cummax()
    df['dd'] = df['cum_roll_max'] - df['cum_ret']
    df['dd_percent'] = df['dd']/df['cum_roll_max']
    max_dd_percent = df['dd_percent'].max()
    return max_dd_percent

def clmr(dataframe):
    df = dataframe.copy()
    clmr = CAGR(df)/max_dd(df)
    return clmr

